In [122]:
import pandas as pd
import numpy as np
import random
import math
from itertools import permutations

In [123]:
# COUNT Completion Time of sequence of order in flowshop schedule
def count_completion_time(processing_times):
    process_accumlutaion = np.zeros(processing_times.shape)
    accumulation_time = 0
    for i in range(len(processing_times[0])):
        accumulation_time += processing_times[0][i]
        process_accumlutaion[0][i] = accumulation_time

    for i in range(1, len(processing_times)):
        accumulation_time = process_accumlutaion[i-1][0]
        for j in range(len(processing_times[0])):
            if(j == 0):
                accumulation_time += processing_times[i][j]
            else:
                if(process_accumlutaion[i-1][j] < process_accumlutaion[i][j-1]):
                    accumulation_time += processing_times[i][j]
                else:
                    accumulation_time = process_accumlutaion[i-1][j]
                    accumulation_time += processing_times[i][j]
            process_accumlutaion[i][j] = accumulation_time
    return accumulation_time

In [124]:
def generate_random_sequence():
  """Generates a random sequence of numbers from 1 to 6."""
  sequence = random.sample(range(1, 7), 6)  # Sample 6 unique numbers from range 1 to 6
  return ''.join(map(str, sequence))  # Convert sequence to string with '-' separator


In [125]:
def sequence_to_string(sequence):
  """Converts a sequence of numbers to a string.
  Args:
      sequence: A tuple of integers representing the sequence.

  Returns:
      A string representation of the sequence.
  """
  return ''.join(map(str, sequence))

In [126]:

def generate_all_possibilites(data):
    numbers = range(1, 7)  # Create a sequence from 1 to 6

    # Use permutations to generate all possible orderings
    all_sequences = list(permutations(numbers))

    # Convert to List of String
    seq = [sequence_to_string(seq) for seq in all_sequences]
    
    sol = dict()
    for x in range(len(seq)):
        seq_list = np.zeros(data.shape)
        for i in range(len(seq[x])):
            seq_list[i] = data[int(seq[x][i])-1]
        sol.update({seq[x]: count_completion_time(seq_list)})
    return sol


In [127]:
# Generate Random Solution of amount_of_solution in return of dictionary with data
# {"sequence" : "compleiton_time"}
# ...
def generate_random_solution(data, amount_of_solution:int):
    sol = dict()
    for x in range(amount_of_solution):
        seq = generate_random_sequence()
        seq_list = np.zeros(data.shape)
        for i in range(len(seq)):
            seq_list[i] = data[int(seq[i])-1]
        sol.update({seq: count_completion_time(seq_list)})
    return sol


In [128]:
def min_max_normalization(arr):
    min_val = np.min(arr) -1 # (np.min(arr)  * 1/100)
    max_val = np.max(arr)
    return (arr - min_val) / (max_val - min_val)

In [129]:
# Z-Score Normalization 
def zscore_normalization(arr):
    mean_val = np.mean(arr)
    std_dev = np.std(arr)
    return (arr - mean_val) / std_dev


In [130]:
def find_up_low_bound(arr):
    max_val = np.max(arr)
    min_val = np.min(arr)
    up_bound = math.ceil(max_val)
    low_bound = math.floor(min_val)
    return low_bound, up_bound

In [131]:
def sort_by_index(data):
  """Sorts a list and returns the corresponding sorted index sequence.

  Args:
      data: A list of sortable elements.

  Returns:
      A list of indices representing the sorted order of the original list.
  """
  return sorted(range(len(data)), key=lambda i: data[i])


### RUN

In [132]:
#LOAD DATA
data = pd.read_csv("data.csv")
data = data.values
data = data[:, 1:]


In [133]:
# INIT THE PARAMETERS
population_size = 10
iteration_ = 50
gamma = 0.01
alpha = 0.1
beta = 0.2
delta = 0.97

In [134]:
# Generate Random Solution based on population_size
solution_ = generate_random_solution(data=data, amount_of_solution=population_size)

In [135]:
# RUN THIS FOR ALL POSSIBILITIES
solution_ = generate_all_possibilites(data=data)

In [136]:
solution_

{'123456': 273.0,
 '123465': 286.0,
 '123546': 273.0,
 '123564': 276.0,
 '123645': 286.0,
 '123654': 278.0,
 '124356': 273.0,
 '124365': 286.0,
 '124536': 273.0,
 '124563': 281.0,
 '124635': 286.0,
 '124653': 281.0,
 '125346': 273.0,
 '125364': 276.0,
 '125436': 273.0,
 '125463': 281.0,
 '125634': 276.0,
 '125643': 281.0,
 '126345': 286.0,
 '126354': 278.0,
 '126435': 286.0,
 '126453': 281.0,
 '126534': 276.0,
 '126543': 281.0,
 '132456': 273.0,
 '132465': 286.0,
 '132546': 273.0,
 '132564': 276.0,
 '132645': 286.0,
 '132654': 278.0,
 '134256': 273.0,
 '134265': 286.0,
 '134526': 273.0,
 '134562': 286.0,
 '134625': 286.0,
 '134652': 286.0,
 '135246': 273.0,
 '135264': 276.0,
 '135426': 273.0,
 '135462': 286.0,
 '135624': 278.0,
 '135642': 286.0,
 '136245': 286.0,
 '136254': 278.0,
 '136425': 286.0,
 '136452': 286.0,
 '136524': 278.0,
 '136542': 286.0,
 '142356': 273.0,
 '142365': 286.0,
 '142536': 273.0,
 '142563': 281.0,
 '142635': 286.0,
 '142653': 281.0,
 '143256': 273.0,
 '143265':

In [137]:
solution_keys = list(solution_.keys())
solution_list = list(solution_.values())
solution_list = np.array(solution_list)

In [138]:
min(solution_list)

273.0

In [139]:
normalize_solution = zscore_normalization(solution_list)

In [140]:
normalize_solution

array([-1.89908148,  0.7088121 , -1.89908148, -1.29725988,  0.7088121 ,
       -0.89604549, -1.89908148,  0.7088121 , -1.89908148, -0.29422389,
        0.7088121 , -0.29422389, -1.89908148, -1.29725988, -1.89908148,
       -0.29422389, -1.29725988, -0.29422389,  0.7088121 , -0.89604549,
        0.7088121 , -0.29422389, -1.29725988, -0.29422389, -1.89908148,
        0.7088121 , -1.89908148, -1.29725988,  0.7088121 , -0.89604549,
       -1.89908148,  0.7088121 , -1.89908148,  0.7088121 ,  0.7088121 ,
        0.7088121 , -1.89908148, -1.29725988, -1.89908148,  0.7088121 ,
       -0.89604549,  0.7088121 ,  0.7088121 , -0.89604549,  0.7088121 ,
        0.7088121 , -0.89604549,  0.7088121 , -1.89908148,  0.7088121 ,
       -1.89908148, -0.29422389,  0.7088121 , -0.29422389, -1.89908148,
        0.7088121 , -1.89908148,  0.7088121 ,  0.7088121 ,  0.7088121 ,
       -1.89908148, -0.29422389, -1.89908148,  0.7088121 , -0.29422389,
        0.7088121 ,  0.7088121 , -0.29422389,  0.7088121 ,  0.70

In [141]:
low_bound, up_bound  = find_up_low_bound(normalize_solution)

In [142]:
# CREATE OBJECT AND RUN ALGORITHM
from firefly import firefly_algo

In [143]:
fa = firefly_algo(data=normalize_solution, population=population_size, max_iter=iteration_, low_bound=low_bound, up_bound=up_bound, gamma=gamma, alpha=alpha, beta=beta, delta=delta)
fa.run()

In [144]:
fa.print_result()

-84.567
0.709
-76.201
-65.165
0.709
-23.781
-53.125
0.709
-39.683
-0.294
0.709
-0.294
-1.899
-1.297
-1.899
-0.294
-1.297
-0.294
0.709
-0.896
0.709
-0.294
-1.297
-0.294
-1.899
0.709
-1.899
-1.297
0.709
-0.896
-1.899
0.709
-1.899
0.709
0.709
0.709
-1.899
-1.297
-1.899
0.709
-0.896
0.709
0.709
-0.896
0.709
0.709
-0.896
0.709
-1.899
0.709
-1.899
-0.294
0.709
-0.294
-1.899
0.709
-1.899
0.709
0.709
0.709
-1.899
-0.294
-1.899
0.709
-0.294
0.709
0.709
-0.294
0.709
0.709
-0.294
0.709
-1.899
-1.297
-1.899
-0.294
-1.297
-0.294
-1.899
-1.297
-1.899
0.709
-0.896
0.709
-1.899
-0.294
-1.899
0.709
-0.294
0.709
-1.297
-0.294
-0.896
0.709
-0.294
0.709
0.709
-0.896
0.709
-0.294
-1.297
-0.294
0.709
-0.896
0.709
0.709
-0.896
0.709
0.709
-0.294
0.709
0.709
-0.294
0.709
-1.297
-0.294
-0.896
0.709
-0.294
0.709
-1.097
1.511
-1.097
-0.495
1.511
-0.094
-1.097
1.511
-1.097
0.508
1.511
0.508
-1.097
-0.495
-1.097
0.508
-0.495
0.508
1.511
-0.094
1.511
0.508
-0.495
0.508
-1.097
1.511
-1.097
-0.495
1.511
-0.094
-1.097

In [145]:
sorted_indices = sort_by_index(normalize_solution)

In [146]:
res = dict()
for i in sorted_indices:
    res.update({solution_keys[i]: fa.data[i]})

In [147]:
res

{'123456': -84.56723201126285,
 '123546': -76.20054502522143,
 '123564': -65.16452135186469,
 '124356': -53.125025588316944,
 '124536': -39.68286500030103,
 '123654': -23.78139722686945,
 '125346': -1.899081478832459,
 '125436': -1.899081478832459,
 '132456': -1.899081478832459,
 '132546': -1.899081478832459,
 '134256': -1.899081478832459,
 '134526': -1.899081478832459,
 '135246': -1.899081478832459,
 '135426': -1.899081478832459,
 '142356': -1.899081478832459,
 '142536': -1.899081478832459,
 '143256': -1.899081478832459,
 '143526': -1.899081478832459,
 '145236': -1.899081478832459,
 '145326': -1.899081478832459,
 '152346': -1.899081478832459,
 '152436': -1.899081478832459,
 '153246': -1.899081478832459,
 '153426': -1.899081478832459,
 '154236': -1.899081478832459,
 '154326': -1.899081478832459,
 '312456': -1.698474280364241,
 '312546': -1.698474280364241,
 '314256': -1.698474280364241,
 '314526': -1.698474280364241,
 '315246': -1.698474280364241,
 '315426': -1.698474280364241,
 '32145